In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
path = Path('/bigdata/f_mnist/logs/baselines/1644945876/rn18_18_dNone_256_adam_l0.001_g0.1_sTrue')

layer = 28
logdir = path / f'features_layer{layer}_seed{seed}'
logdir.mkdir(parents=True, exist_ok=True)


baseline_path = Path('/bigdata/f_mnist/logs/baselines/1644945876/rn18_18_dNone_256_adam_l0.001_g0.1_sTrue/cnn_best.pth')
baseline_model = torch.load(path / "cnn_best.pth")

In [ ]:
results = []
for path in paths:
    with open(path / 'results_approximation_error.json') as json_file:
        result = json.load(json_file)

    with open(path / 'results_feature_metrics.json') as json_file:
        feature_result = json.load(json_file)
    
    result.update(feature_result['mean'])
    # print(result)
    for k,v in result.items():
        result[k] = v[0] if type(v) == list else v
    results.append(result)

In [ ]:
df = pd.DataFrame(results)
df.head()


In [ ]:
baseline_path = Path("/bigdata/cifar10/logs/baselines/1646668631/rn18_18_dNone_128_adam_l0.001_g0.1_w0.0_sTrue")
with open(baseline_path/'results.json') as json_file:
    baseline_result = json.load(json_file)

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")

In [ ]:
ax = sns.lineplot(x="rank", y="relative_norm",
    hue="factorization",
    # style="rank",
    data=df,
    palette=['orange', 'b'],
)
ax.set(xlabel='relative rank', ylabel='relative error on features: |B-F| / |B|')

In [ ]:
ax = sns.scatterplot(x="relative_norm", y="valid_acc_before_ft",
    hue="factorization",
    # style="rank",
    data=df,
    palette=['orange', 'b'],
)
ax.axhline(baseline_result['best_valid_acc'], color='r')
ax.set(xlabel='relative error on features: |B-F| / |B|', ylabel='Validation accuracy before fine-tuning')
ax.set(ylim=(0.865, 0.925))

In [ ]:
ax = sns.scatterplot(x="relative_norm", y="valid_acc",
    hue="factorization",
    # style="rank",
    data=df,
    palette=['orange', 'b'],
)
ax.axhline(baseline_result['best_valid_acc'], color='r')
ax.set(xlabel='relative error on features: |B-F| / |B|', ylabel='Validation accuracy after fine-tuning')
ax.set(ylim=(0.865, 0.925))

In [ ]:
ax = sns.scatterplot(x="relative_norm", y="test_acc",
    hue="factorization",
    # style="rank",
    data=df,
    palette=['orange', 'b'],
)
ax.axhline(baseline_result['test_acc'], color='g')
ax.set(xlabel='relative error on features: |B-F| / |B|', ylabel='Test accuracy after fine-tuning')
ax.set(ylim=(0.865, 0.925))

In [2]:
from tqdm import tqdm
from tddl.features.extract import aggregate_results
import os
from pathlib import Path

path = Path("/bigdata/cifar10/logs/decomposed/")

for nr in tqdm(os.listdir(path)):
    path_nr = path / nr
    dirs = [d for d in os.listdir(path_nr) if os.path.isdir(path_nr / d)]
    path_modelname = path_nr / dirs[0]

    path_features = path_modelname / 'features' 
    path_split = path_features / 'train'

    try:
        path_split.mkdir(exist_ok=False)
    except:
        ...
    # files = os.listdir(path_features)
    file_paths = [(path_features / f, path_split / f) for f in os.listdir(path_features) if not (path_features / f).is_dir()]
    print(file_paths)
    break
    # aggregate_results(path_modelname / dirs[0], split='train')

  0%|          | 0/120 [00:01<?, ?it/s]